In [ ]:
%matplotlib inline
import datetime
import matplotlib.pyplot as plt
import glob
import os
from obspy import read, UTCDateTime, read_inventory
from obspy.signal import PPSD
import warnings
import pandas as pd
import numpy as np
import matplotlib.gridspec as gridspec
from matplotlib.dates import DateFormatter
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
plt.style.use("ggplot")

from msnoise.api import *

In [ ]:
# connect to the database
db = connect()
params = get_params(db)

In [ ]:
for filter in get_filters(db):
    filter.used = False
db.commit()
update_filter(db, ref=5, low=0.5, mwcs_low=0.5, high=8, mwcs_high=8, mwcs_wlen=4, mwcs_step=1, used=1)


In [ ]:
! msnoise info

In [ ]:
! msnoise config set mov_stack=(('1d','1d'),('2d','1d'),('12h','3h'))
! msnoise config set components_to_compute_single_station=EN,EZ,NZ
! msnoise config set components_to_compute=""
! msnoise reset DA
! msnoise new_jobs
! msnoise db execute "update jobs set flag='D' where jobtype='CC'"
! msnoise db execute "update jobs set flag='T' where jobtype='CC' and day between '2014-06-01' and '2014-07-01'"

In [ ]:
! msnoise info -j

In [ ]:
! msnoise -q -t 6 cc compute_cc

In [ ]:
# Obtain a list of dates between ``start_date`` and ``enddate``
start, end, datelist = build_movstack_datelist(db)

# Get the list of parameters from the DB:
params = get_params(db)

# Get the time axis for plotting the CCF:
taxis = get_t_axis(db)

filter_id = 5
comp = "EN"
# Get the results for two station, filter id=1, ZZ component, mov_stack=("1d","1d") and the results as a 2D array:
ccfs = get_results_all(db, "PF.CSS.00", "PF.CSS.00", filter_id, "EN", datelist, format="xarray")


In [ ]:
ccfs.CCF.plot(figsize=(12,6))

In [ ]:
zoom = 20 # seconds in the coda

In [ ]:
ccfs.CCF.loc[:,-zoom:zoom].plot(figsize=(12,6), vmin=-1e-6)

In [ ]:
ccfs.CCF.loc[:,-zoom:zoom].rolling(times=24).mean().resample(times='3h').mean().plot(figsize=(12,6), vmin=-5e-7)

In [ ]:
! msnoise -q -t 6 cc stack -r

In [ ]:
! msnoise reset STACK
! msnoise -q -t 6 cc stack -m

In [ ]:
# ! msnoise db execute "update jobs set flag='T' where jobtype='MWCS' and day between '2014-06-01' and '2014-07-01'"
! msnoise -q -t 5 cc dvv compute_mwcs

In [ ]:
! msnoise -q -t 5 cc dvv compute_dtt

In [ ]:
! msnoise -q cc dvv compute_dvv

In [ ]:
from msnoise.plots.dvv import main as plot_dvv
for comp in ["EN", "EZ", "NZ"]:
    plot_dvv(filterid=5, components=comp, show=False)
    for ax in plt.gcf().axes:
        plt.sca(ax)
        plt.axvline(datetime.datetime(2014,6,21,),c='r', ls='--')
    plt.show()

In [ ]:
# plot_dvv(filterid=5, components="EZ,NZ,EN", show=False)
# for ax in plt.gcf().axes:
#     plt.sca(ax)
#     plt.axvline(datetime.datetime(2014,6,21,),c='r', ls='--')
# plt.show()

In [ ]:
fig, axes = plt.subplots(3,1, figsize=(10,10))
filterid = 5
mov_stack = params.mov_stack[-1]
plt.suptitle(mov_stack)
filter = get_filters(db, ref=filterid)
for i, comp in enumerate(["EN", "EZ", "NZ"]):
    for sta1, sta2 in get_station_pairs(db):
        plt.sca(axes[i])
        for loc1 in sta1.locs():
            s1 = "%s.%s.%s" % (sta1.net, sta1.sta, loc1)
            for loc2 in sta2.locs():
                s2 = "%s.%s.%s" % (sta2.net, sta2.sta, loc2)
                if s1 != s2:
                    continue
                dvv = compute_dvv(db, filterid, mov_stack, components=comp, params=params, pairs=[(s1,s2),])
                dvv.m0 *= -100
                # dvv = dvv.rolling("24h", closed="right", center=False).mean()
                dvv.m0.plot(label="%s:%s"%(s1,s2), ax=axes[i])
                plt.title("Filter %i (%.2f - %.2f Hz) - %s" % (filterid, filter.low, filter.high, comp))
        plt.axvline(datetime.date(2014,6,21), c='r', ls='--')
        plt.legend()
        plt.xlabel("")
        plt.ylabel("dv/v (%)")
plt.tight_layout()

In [ ]:
dvv

Wavelet Method